In [1]:
from qililab.controllers.qblox_cluster_controller import QbloxClusterController
from qililab.controllers.qdevil_qdac2_controller import QDevilQDAC2Controller
from qililab.runcard.runcard import Runcard
from qililab.settings.controllers.qblox_cluster_controller_settings import (
    QbloxClusterControllerSettings,
    QbloxClusterModule,
)
from qililab.settings.controllers.qdevil_qdac2_controller_settings import QDevilQDAC2ControllerSettings

2025-06-25 18:07:09,189 - qm - INFO     - Starting session: 24f5f700-4998-424d-8393-1629aded124c


In [2]:
qdac2 = QDevilQDAC2Controller(settings=QDevilQDAC2ControllerSettings(alias="qdac2"))
qdac2

QDevilQDAC2Controller(settings=alias='qdac2' connection=ConnectionSettings(type=<ConnectionType.TCP_IP: 'tcp_ip'>, address='168.0.0.1') reset=True)

In [3]:
cluster = QbloxClusterController(
    settings=QbloxClusterControllerSettings(
        alias="cluster", modules={0: QbloxClusterModule.QCM, 1: QbloxClusterModule.QRM}
    )
)

In [4]:
runcard = Runcard(name="my_lab")
runcard

Runcard(name='my_lab', controllers=[])

In [5]:
runcard.add_controller(qdac2)
runcard.add_controller(cluster)
runcard

Runcard(name='my_lab', controllers=[QDevilQDAC2RuncardController(type=<ControllerType.QDEVIL_QDAC2_CONTROLLER: 'qdac2_controller'>, settings=QDevilQDAC2ControllerSettings(alias='qdac2', connection=ConnectionSettings(type=<ConnectionType.TCP_IP: 'tcp_ip'>, address='168.0.0.1'), reset=True)), QbloxClusterRuncardController(type=<ControllerType.QBLOX_CLUSTER_CONTROLLER: 'qblox_cluster_controller'>, settings=QbloxClusterControllerSettings(alias='cluster', connection=ConnectionSettings(type=<ConnectionType.TCP_IP: 'tcp_ip'>, address='168.0.0.1'), reset=True, modules={0: <QbloxClusterModule.QCM: 'QCM'>, 1: <QbloxClusterModule.QRM: 'QRM'>}))])

In [6]:
runcard.save_to("runcard.yml")

In [7]:
loaded_runcard = Runcard.load_from("runcard.yml")
loaded_runcard

Runcard(name='my_lab', controllers=[QDevilQDAC2RuncardController(type=<ControllerType.QDEVIL_QDAC2_CONTROLLER: 'qdac2_controller'>, settings=QDevilQDAC2ControllerSettings(alias='qdac2', connection=ConnectionSettings(type=<ConnectionType.TCP_IP: 'tcp_ip'>, address='168.0.0.1'), reset=True)), QbloxClusterRuncardController(type=<ControllerType.QBLOX_CLUSTER_CONTROLLER: 'qblox_cluster_controller'>, settings=QbloxClusterControllerSettings(alias='cluster', connection=ConnectionSettings(type=<ConnectionType.TCP_IP: 'tcp_ip'>, address='168.0.0.1'), reset=True, modules={0: <QbloxClusterModule.QCM: 'QCM'>, 1: <QbloxClusterModule.QRM: 'QRM'>}))])

In [8]:
from qililab.settings.port import IntPort, ModulePort, NoPort, Port

In [9]:
from typing import ClassVar, Generic, Type, TypeVar

from pydantic import BaseModel

P = TypeVar("P", bound=Port)  # “shape of the port”


class Instrument(BaseModel, Generic[P]):
    """
    Base class for all instruments.

    Sub-classes must override the `Port` ClassVar with the model that
    describes *their* port shape.
    """

    Port: ClassVar[Type[P]]  # <- not a Pydantic field
    name: str

    # -------- default validator (may be overridden) ------------------
    @classmethod
    def validate_port(cls, port: P) -> P:
        """Override this to enforce instrument-specific constraints."""
        return port  # default: accept anything


# ---- concrete instruments --------------------------------------------------


class Thermometer(Instrument[IntPort]):
    Port = IntPort

    @classmethod
    def validate_port(cls, port: IntPort) -> IntPort:
        if not 0 <= port.id <= 32:
            raise ValueError("Thermometer: id must be 0-32")
        return port  # simple integer port


class Spectrometer(Instrument[ModulePort]):
    Port = ModulePort  # module, channel

    @classmethod
    def validate_port(cls, port: ModulePort) -> ModulePort:
        if not 0 <= port.module <= 31:
            raise ValueError("Spectrometer has modules 0-31")
        if not 1 <= port.port <= 4:
            raise ValueError("Spectrometer's modules has ports 1-4")
        return port


class Pump(Instrument[NoPort]):
    Port = NoPort  # no port at all

In [10]:
from pydantic import model_validator

I = TypeVar("I", bound=Instrument)


class Bus(BaseModel):
    name: str
    address: str


class Association(BaseModel, Generic[I, P]):
    """
    An instrument wired to a bus **through** a particular port value.
    """

    bus: Bus
    instrument: I
    port: P

    # --- runtime check that `port` matches `instrument.Port` ----------
    @model_validator(mode="after")
    def check_port_matches_instrument(self):
        expected_model = self.instrument.Port
        if not isinstance(self.port, expected_model):
            raise ValueError(
                f"Port must be of type {expected_model.__name__} " f"for {self.instrument.__class__.__name__}"
            )

        # 2) Delegate value checks to the instrument -------------------------
        self.instrument.__class__.validate_port(self.port)
        return self

In [11]:
ThermoAssoc = Association[Thermometer, IntPort]
SpectroAssoc = Association[Spectrometer, ModulePort]
PumpAssoc = Association[Pump, NoPort]

In [12]:
# ────────────────────────────────────────────
bus1 = Bus(name="Main PLC", address="192.168.0.10")

T1 = Thermometer(name="T-1")
OK = Association(bus=bus1, instrument=T1, port=IntPort(id=3))  # ✅

S1 = Spectrometer(name="S-1")
OK2 = Association(bus=bus1, instrument=S1, port=ModulePort(module=2, port=0))  # ✅

# --- type-checker will reject this (and runtime will raise) ----------
BAD = Association(bus=bus1, instrument=S1, port=IntPort(id=99))  # ❌ mypy + ValidationError

ValidationError: 1 validation error for Association
  Value error, Spectrometer's modules has ports 1-4 [type=value_error, input_value={'bus': Bus(name='Main PL...ule', module=2, port=0)}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

In [ ]:
# ────────────────────────────────────────────
bus1 = Bus(name="Main PLC", address="192.168.0.10")

T1 = Thermometer(name="T-1")
OK = ThermoAssoc(bus=bus1, instrument=T1, port=IntPort(id=3))  # ✅

S1 = Spectrometer(name="S-1")
OK2 = SpectroAssoc(bus=bus1, instrument=S1, port=ModulePort(module=2, port=0))  # ✅

# --- type-checker will reject this (and runtime will raise) ----------
BAD = SpectroAssoc(bus=bus1, instrument=S1, port=IntPort(id=99))  # ❌ mypy + ValidationError

ValidationError: 1 validation error for Association[Spectrometer, ModulePort]
port
  Input should be a valid dictionary or instance of ModulePort [type=model_type, input_value=IntPort(kind='int', id=99), input_type=IntPort]
    For further information visit https://errors.pydantic.dev/2.11/v/model_type